In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
a=pd.read_csv("../input/train.csv")
df=pd.DataFrame(a)
xtrain=df[df.columns[1:]]
xtrain=np.array(xtrain)
Xtrain=xtrain.reshape(42000,28,28,1)
ytrain=df[df.columns[0]]
ytrain[2]

1

In [3]:
from keras.models import Model
from keras.layers import (
    Input,
    Dense,
    Flatten,
    Dropout
)
from keras.layers.convolutional import (
    AveragePooling2D,
    Conv2D,
    MaxPooling2D
)
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU, LeakyReLU, ELU, ThresholdedReLU
from keras.regularizers import l2
from keras import backend as K
from keras.layers.core import Reshape
from keras import regularizers
from keras.layers.merge import concatenate

class model_cnn(object):
    @staticmethod
    def build_model(input_shape):
        batch_size = 128
        num_classes = 10
        epochs = 12
        input = Input(shape=input_shape)
        
        x10 = Conv2D(kernel_initializer='he_normal', strides=(1, 1), kernel_regularizer=regularizers.l2(0.0001),
                      filters=4, kernel_size=(3,3), padding='valid')(input)
        
        x12 = Conv2D(kernel_initializer='he_normal', strides=(1, 1), kernel_regularizer=regularizers.l2(0.0001),
                      filters=3, kernel_size=(3,3), padding='valid')(x10)
        x121 = Conv2D(kernel_initializer='he_normal', strides=(1, 1), kernel_regularizer=regularizers.l2(0.0001),
                      filters=3, kernel_size=(3,3), padding='valid')(x12)
        x122 = Conv2D(kernel_initializer='he_normal', strides=(1, 1), kernel_regularizer=regularizers.l2(0.0001),
                      filters=3, kernel_size=(3,3), padding='valid')(x121)
        x123 = Conv2D(kernel_initializer='he_normal', strides=(1, 1), kernel_regularizer=regularizers.l2(0.0001),
                      filters=3, kernel_size=(3,3), padding='valid')(x122)
        x124 = Conv2D(kernel_initializer='he_normal', strides=(1, 1), kernel_regularizer=regularizers.l2(0.0001),
                      filters=3, kernel_size=(3,3), padding='valid')(x123)
        
        x12 = Conv2D(kernel_initializer='he_normal', strides=(1, 1), kernel_regularizer=regularizers.l2(0.0001),
                      filters=3, kernel_size=(3,3), padding='valid')(x124)
        
        x13 = Conv2D(kernel_initializer='he_normal', strides=(1, 1), kernel_regularizer=regularizers.l2(0.0001),
                      filters=4, kernel_size=(3,3), padding='valid')(x12)
        drop00=Dropout(0.5)(x13)
        
        x1 = Conv2D(kernel_initializer='he_normal', strides=(1, 1), kernel_regularizer=regularizers.l2(0.0001),
                      filters=4, kernel_size=(3,3), padding='valid')(drop00)
        
        x2=AveragePooling2D(pool_size=(x1._keras_shape[1], x1._keras_shape[2]), strides=(1, 1))(x1)
        pool1 = AveragePooling2D(pool_size=(x2._keras_shape[1], x2._keras_shape[2]), strides=(1, 1))(x2)

        flatten1 = Flatten()(pool1)
        drop1 = Dropout(0.5)(flatten1)
        dense = Dense(units=10, activation="softmax", kernel_initializer="glorot_normal")(drop1)

        model = Model(inputs=input, outputs=dense)        
        return model





Using TensorFlow backend.


In [4]:
from keras.utils import to_categorical
ytrain=to_categorical (ytrain)


In [5]:
from keras.optimizers import RMSprop
model = model_cnn.build_model([28,28,1])
rms = RMSprop(lr=0.0003)
model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
model.fit(Xtrain, ytrain, epochs=15)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
42000/42000 [==============================] - 43s 1ms/step - loss: 13.0605 - acc: 0.1045
Epoch 2/15
42000/42000 [==============================] - 41s 987us/step - loss: 9.4315 - acc: 0.1437
Epoch 3/15
42000/42000 [==============================] - 41s 985us/step - loss: 2.1097 - acc: 0.2535
Epoch 4/15
42000/42000 [==============================] - 42s 1ms/step - loss: 1.8136 - acc: 0.3150
Epoch 5/15
42000/42000 [==============================] - 42s 988us/step - loss: 1.7216 - acc: 0.3370
Epoch 6/15
42000/42000 [==============================] - 41s 986us/step - loss: 1.6764 - acc: 0.3545
Epoch 7/15
42000/42000 [==============================] - 42s 994us/step - loss: 1.6406 - acc: 0.3699
Epoch 8/15
42000/42000 [==============================

In [6]:
ytrain[42000-1-1]


array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [7]:
a=pd.read_csv("../input/test.csv")
df=pd.DataFrame(a)
xtest=df[df.columns[0:]]

xtest=np.array(xtest)
Xtest=xtest.reshape(28000,28,28,1)


n=model.predict(Xtest).argmax(axis=1)


In [8]:
len(n)

28000

In [9]:
k2=model.predict(Xtrain).argmax(axis=1)

In [10]:
sub=pd.DataFrame(
                    {
                    "ImageId":list(range(1,28001)),
                        "Label":n})
sub

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,4
4,5,2
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [11]:
sub.to_csv("submission2.csv",index=False)
